In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import pymc as pm
import arviz as az

%load_ext lab_black

# Pumps

Adapted from [Unit 5: pumpsmc.m](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit5/pumpsmc.m) and [pumpsbugs.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit5/pumpsbugs.odc).

Associated lecture video: Unit 5 lesson 13

Based on [Conjugate Likelihood Distributions, George, Makov, and Smith, 1993](https://www.jstor.org/stable/4616270) and [Robust Empirical Bayes Analyses of Event Rates, Gaver and O'Muircheartaigh, 1987](https://www.jstor.org/stable/1269878).

In [2]:
rng = np.random.default_rng(1)

obs = 100000
burn = 1000

# data from Gaver and O'Muircheartaigh, 1987
X = np.array([5, 1, 5, 14, 3, 19, 1, 1, 4, 22])
t = np.array([94.32, 15.52, 62.88, 125.76, 5.24, 31.44, 1.048, 1.048, 2.096, 10.48])
n = len(X)

# params
c = 0.1
d = 1

# inits
theta = np.ones(n)
beta = 1

thetas = np.zeros((obs, 10))
betas = np.zeros(obs)

for i in tqdm(range(obs)):
    theta = rng.gamma(shape=X + 1, scale=1 / (beta + t), size=n)
    sum_theta = np.sum(theta)

    beta = rng.gamma(shape=n + c, scale=1 / (sum_theta + d))

    thetas[i] = theta
    betas[i] = beta

thetas = thetas[burn:]
betas = betas[burn:]

print(f"{np.mean(thetas, axis=0)=}")
print(f"{np.std(thetas)=}")
print(f"{np.mean(betas)=}")
print(f"{np.std(betas)=}")

  0%|          | 0/100000 [00:00<?, ?it/s]

np.mean(thetas, axis=0)=array([0.06283613, 0.11838764, 0.0934282 , 0.11795504, 0.61044286,
       0.61054294, 0.87096223, 0.87103813, 1.48272156, 1.94964544])
np.std(thetas)=0.7294065045864946
np.mean(betas)=1.3367177163632642
np.std(betas)=0.4843747815427844


In [3]:
np.mean(thetas, axis=0)

array([0.06283613, 0.11838764, 0.0934282 , 0.11795504, 0.61044286,
       0.61054294, 0.87096223, 0.87103813, 1.48272156, 1.94964544])

Now for the PyMC model:

In [4]:
with pm.Model() as m:
    times = pm.Data("times", t)
    data = pm.Data("data", X)

    beta = pm.Gamma("beta", alpha=c, beta=d)
    theta = pm.Gamma("theta", alpha=d, beta=beta, shape=n)
    rate = pm.Deterministic("rate", theta * times)

    likelihood = pm.Poisson("likelihood", mu=rate, observed=data)

    # start sampling
    trace = pm.sample(5000)

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/data.py:427: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, theta]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 3 seconds.
Chain <xarray.DataArray 'chain' ()>
array(0)
Coordinates:
    chain    int64 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(1)
Coordinates:
    chain    int64 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(2)
Coordinates:
    chain    int64 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain <xarray.DataArray 'chain' ()>
array(3)
Coordinates:
    chain    int64 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.


In [5]:
az.summary(trace, var_names=["~rate"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta,1.337,0.487,0.511,2.235,0.003,0.002,27853.0,17343.0,1.0
theta[0],0.063,0.026,0.019,0.111,0.000,0.000,29657.0,14437.0,1.0
theta[1],0.118,0.083,0.004,0.269,0.000,0.000,25205.0,12218.0,1.0
theta[2],0.094,0.038,0.030,0.164,0.000,0.000,28528.0,13043.0,1.0
theta[3],0.118,0.031,0.062,0.175,0.000,0.000,31702.0,14191.0,1.0
theta[4],0.609,0.309,0.109,1.173,0.002,0.001,26039.0,12618.0,1.0
theta[5],0.611,0.136,0.363,0.870,0.001,0.001,31196.0,14567.0,1.0
theta[6],0.871,0.653,0.017,2.031,0.004,0.003,24217.0,13140.0,1.0
theta[7],0.869,0.652,0.021,2.058,0.004,0.003,25976.0,13618.0,1.0
theta[8],1.482,0.704,0.341,2.770,0.004,0.003,29738.0,13518.0,1.0


Compare to BUGS results:

|           | mean    | sd      | MC_error | val2.5pc | median  | val97.5pc | start | sample |
|-----------|---------|---------|----------|----------|---------|-----------|-------|--------|
| beta      | 1.34    | 0.486   | 0.002973 | 0.5896   | 1.271   | 2.466     | 1001  | 50000  |
| theta[1]  | 0.06261 | 0.02552 | 1.11E-04 | 0.02334  | 0.05914 | 0.1217    | 1001  | 50000  |
| theta[2]  | 0.118   | 0.08349 | 3.69E-04 | 0.01431  | 0.09888 | 0.3296    | 1001  | 50000  |
| theta[3]  | 0.09366 | 0.03829 | 1.71E-04 | 0.03439  | 0.08842 | 0.1828    | 1001  | 50000  |
| theta[4]  | 0.1178  | 0.03048 | 1.47E-04 | 0.06595  | 0.115   | 0.1848    | 1001  | 50000  |
| theta[5]  | 0.6116  | 0.3097  | 0.001409 | 0.1632   | 0.5589  | 1.361     | 1001  | 50000  |
| theta[6]  | 0.6104  | 0.1366  | 6.45E-04 | 0.3705   | 0.6001  | 0.9058    | 1001  | 50000  |
| theta[7]  | 0.8686  | 0.6494  | 0.003059 | 0.101    | 0.7124  | 2.537     | 1001  | 50000  |
| theta[8]  | 0.8692  | 0.6481  | 0.003354 | 0.09784  | 0.7117  | 2.513     | 1001  | 50000  |
| theta[9]  | 1.478   | 0.6897  | 0.00351  | 0.4705   | 1.367   | 3.128     | 1001  | 50000  |
| theta[10] | 1.944   | 0.4133  | 0.002022 | 1.223    | 1.916   | 2.83      | 1001  | 50000  |

In [6]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Sat Mar 18 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

numpy     : 1.24.2
matplotlib: 3.6.3
arviz     : 0.14.0
pymc      : 5.1.1

